## Logistic regression

This notebook compares various logistic regression implementations. The dataset used is the [New South Wales electricity price dataset](https://creme-ml.github.io/generated/creme.datasets.fetch_electricity.html#creme.datasets.fetch_electricity).

In [18]:
%load_ext watermark
%watermark --python --machine --packages creme,keras,sklearn,tensorflow,torch --datename

Tue Aug 20 2019 

CPython 3.7.3
IPython 7.4.0

creme 0.3.0
keras 2.2.4
sklearn 0.20.3
tensorflow 1.14.0
torch 1.1.0

compiler   : GCC 7.3.0
system     : Linux
release    : 5.1.9-arch1-1-ARCH
machine    : x86_64
processor  : 
CPU cores  : 8
interpreter: 64bit


In [16]:
from creme import datasets
from creme import linear_model
from creme import metrics
from creme import optim
from creme import preprocessing
from keras import layers
from keras import models
from keras import optimizers
from sklearn import linear_model as sk_linear_model
import torch

%run utils.py
%run wrappers.py

In [17]:
n_features = 8
lr = 0.005

# PyTorch
class PyTorchNet(torch.nn.Module):
    
    def __init__(self, n_features):
        super().__init__()
        self.linear = torch.nn.Linear(n_features, 1)
        self.sigmoid = torch.nn.Sigmoid()
        torch.nn.init.constant_(self.linear.weight, 0)
        torch.nn.init.constant_(self.linear.bias, 0)
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))
    
torch_model = PyTorchNet(n_features=n_features)

# Keras
inputs = layers.Input(shape=(n_features,))
predictions = layers.Dense(1, activation='sigmoid', kernel_initializer='zeros', bias_initializer='zeros')(inputs)
keras_model = models.Model(inputs=inputs, outputs=predictions)
keras_model.compile(optimizer=optimizers.SGD(lr=lr), loss='binary_crossentropy')


def add_hour(x):
    x['hour'] = x['moment'].hour
    return x

results = benchmark(
    get_X_y=datasets.fetch_electricity,
    n=45312,
    get_pp=preprocessing.StandardScaler,
    models=[
        ('creme', 'LogisticRegression', linear_model.LogisticRegression(
            optimizer=optim.SGD(lr),
            l2=0.,
            intercept_lr=lr
        )),

        ('scikit-learn', 'SGDClassifier', ScikitLearnClassifier(
            model=sk_linear_model.SGDClassifier(
                loss='log',
                learning_rate='constant',
                eta0=lr,
                penalty='none'
            ),
            classes=[False, True]
        )),
        
        ('PyTorch (CPU)', 'Linear', PyTorchBinaryClassifier(
            network=torch_model,
            loss_fn=torch.nn.BCELoss(),
            optimizer=torch.optim.SGD(torch_model.parameters(), lr=lr)
        )),
        
        ('Keras on Tensorflow (CPU)', 'Dense', KerasBinaryClassifier(
            model=keras_model
        )),
        
    ],
    get_metric=metrics.LogLoss
)

In [14]:
results

,Library,Model,LogLoss,Fit time,Average fit time,Predict time,Average predict time
0,creme,LogisticRegression,0.413533,"704ms, 19μs, 145ns","15μs, 537ns","243ms, 295μs, 847ns","5μs, 369ns"
1,scikit-learn,SGD,0.413533,"15s, 461ms, 23μs, 898ns","341μs, 213ns","6s, 470ms, 878μs, 136ns","142μs, 807ns"
2,PyTorch (CPU),Linear,0.418901,"38s, 689ms, 746μs, 772ns","853μs, 852ns","14s, 359ms, 296μs, 803ns","316μs, 898ns"
3,Keras on Tensorflow (CPU),Linear,0.418901,"1m, 16s, 304ms, 341μs, 400ns","1ms, 683μs, 976ns","39s, 700ms, 439μs, 782ns","876μs, 157ns"
